### see acoustic_multichannel for downloading data (migrate to README later)

In [1]:
import tensorflow as tf
import numpy as np

import sys
import os

sys.path.insert(0, 'models/research/audioset/vggish/')

import vggish_input
import vggish_params
import vggish_postprocess
import vggish_slim
from sklearn.model_selection import train_test_split

2024-04-23 21:23:31.212564: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 21:23:31.218360: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-23 21:23:31.276047: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 21:23:32.619450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Charger le modèle VGGish
vgg_graph = tf.Graph()
with vgg_graph.as_default():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    vggish_slim.define_vggish_slim(training=True)
    vggish_slim.load_vggish_slim_checkpoint(sess, 'models/research/audioset/vggish//vggish_model.ckpt')

/opt/mamba/lib/python3.11/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/opt/mamba/lib/python3.11/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:318: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '


INFO:tensorflow:Restoring parameters from models/research/audioset/vggish//vggish_model.ckpt


2024-04-23 20:18:46.187696: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


In [76]:
def extract_vggish_features(data):
    features = []
    with vgg_graph.as_default():
        for rir in data:
            #log_mel_spectrogram(rir)
            spectrogram = waveform_to_examples(rir, vggish_params.SAMPLE_RATE)
            features_tensor = sess.graph.get_tensor_by_name(vggish_params.INPUT_TENSOR_NAME)
            embedding_tensor = sess.graph.get_tensor_by_name(vggish_params.OUTPUT_TENSOR_NAME)
            embedding = sess.run(embedding_tensor, feed_dict={features_tensor: spectrogram})
            features.append(embedding)
        print(spectrogram)
        #print(features)
    return np.array(features)

In [2]:
# Charger les données d'entrée (RIRs) et de sortie (positions reconnues) depuis les fichiers .npy
input_data = np.load('data/LivingRoom_preprocessed_hack/Human1/deconvoled_trim.npy')
output_data = np.load('data/LivingRoom_preprocessed_hack/Human1/centroid.npy')

# Diviser les données en ensembles d'entraînement et de test
train_input, test_input, train_output, test_output = train_test_split(input_data, output_data, test_size=0.2, random_state=42)

In [6]:
test_output.shape

(200, 2)

In [15]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(800,800,128)),
        tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])



model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',      
              metrics=['accuracy'])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization

In [1]:
#test = model.fit(input_data, epochs=5)

In [92]:
train_features = extract_vggish_features(train_input)
test_features = extract_vggish_features(test_input)

(array([[ 0.00000000e+00, -6.65863739e-11, -1.84332619e-10, ...,
                    nan,  1.29150466e-42,  3.34225772e-43],
       [ 0.00000000e+00,  2.82027808e-45,  2.16080063e-46, ...,
         2.53783278e-44,  2.14351423e-47,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.22286703e+12, ...,
         1.70200002e-44,  1.29357438e+16,  2.82027808e-45]]), 512)
(0, 96, 64)
(array([[-0.00000000e+00,  1.59691364e-11,  1.87500310e-10, ...,
                    nan,  1.29150466e-42,  3.34225772e-43],
       [ 0.00000000e+00,  2.82027808e-45,  2.16080063e-46, ...,
         2.53783278e-44,  2.14351423e-47,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.22286703e+12, ...,
         1.70200002e-44,  1.29357438e+16,  2.82027808e-45]]), 512)
(0, 96, 64)
(array([[0.00000000e+00, 1.29722631e-10, 4.61159732e-10, ...,
                   nan, 1.29150466e-42, 3.34225772e-43],
       [0.00000000e+00, 2.82027808e-45, 2.16080063e-46, ...,
        2.53783278e-44, 2.14351423e-

(0, 96, 64)
(array([[-0.00000000e+00,  1.58835936e-10,  1.09773159e-09, ...,
                    nan,  1.29150466e-42,  3.34225772e-43],
       [ 0.00000000e+00,  2.82027808e-45,  2.16080063e-46, ...,
         2.53783278e-44,  2.14351423e-47,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.22286703e+12, ...,
         1.70200002e-44,  1.29357438e+16,  2.82027808e-45]]), 512)
(0, 96, 64)
(array([[ 0.00000000e+00,  5.44896967e-11, -4.19989878e-10, ...,
                    nan,  1.29150466e-42,  3.34225772e-43],
       [ 0.00000000e+00,  2.82027808e-45,  2.16080063e-46, ...,
         2.53783278e-44,  2.14351423e-47,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.22286703e+12, ...,
         1.70200002e-44,  1.29357438e+16,  2.82027808e-45]]), 512)
(0, 96, 64)
(array([[ 0.00000000e+00, -4.67234668e-10,  6.19811183e-10, ...,
                    nan,  1.29150466e-42,  3.34225772e-43],
       [ 0.00000000e+00,  2.82027808e-45,  2.16080063e-46, ...,
         2.5378

### modèle

In [53]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(800,800,128)),
        tf.keras.layers.Conv2D(64, [3, 3], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])


model.compile(optimizer='adam', loss='mse')


model.fit(train_features, train_output, epochs=10, batch_size=32, validation_data=(test_features, test_output))


loss = model.evaluate(test_features, test_output)
print("Perte sur les données de test :", loss)

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 0, 128), dtype=float32). Expected shape (None, 800, 800, 128), but input has incompatible shape (32, 0, 128)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 0, 128), dtype=float32)
  • training=True
  • mask=None

In [55]:
train_output.shape

(800, 2)

In [59]:
train_features.shape

(800, 0, 128)

In [57]:
new_train_features = train_features.reshape(800, 800, 128)
new_test_features = test_features.reshape(200, 200, 128)

ValueError: cannot reshape array of size 0 into shape (800,800,128)